In [1]:
require("/home/amit/Downloads/SemVII/QuBase.jl/src/QuBase.jl")
using QuBase
require("/home/amit/Downloads/SemVII/QuDynamics.jl/src/QuDynamics.jl")
using QuDynamics
using BenchmarkLite

In [2]:
type SampleProc{Alg} <: Proc end

In [3]:
type JaynesCummings
    N::Int
    wc::Float64
    wa::Float64
    g::Float64
    kappa::Float64
    gamma::Float64
    use_rwa::Bool
    tlist
end

In [4]:
Base.string{Alg}(::SampleProc{Alg}) = lowercase(string(Alg))

string (generic function with 9 methods)

In [5]:
Base.isvalid{Alg}(p::SampleProc{Alg}, cfg::JaynesCummings) = true

isvalid (generic function with 23 methods)

In [6]:
function operator_set(p::SampleProc, cfg::JaynesCummings)
    idc = QuArray(eye(cfg.N))
    ida = QuArray(eye(2))
    a  = tensor(lowerop(cfg.N), ida)
    sm = tensor(idc, lowerop(2))
    if cfg.use_rwa
        H = cfg.wc * a' * a + cfg.wa * sm' * sm + cfg.g * (a' * sm + a * sm')
    else
        H = cfg.wc * a' * a + cfg.wa * sm' * sm + cfg.g * (a' + a) * (sm + sm')
    end
    c_op_list = Array(QuBase.AbstractQuMatrix, 0)
    n_th_a = 0.0
    rate_1 = cfg.kappa * (1 + n_th_a)
    if rate_1 > 0.0
        push!(c_op_list, full(sqrt(rate_1) * a))
    end
    rate = cfg.kappa * n_th_a
    if rate > 0.0
        push!(c_op_list, full(sqrt(rate) * a'))
    end
    rate = cfg.gamma
    if rate > 0.0
        push!(c_op_list, full(sqrt(rate) * sm))
    end
    psi = complex(tensor(statevec(1, FiniteBasis(cfg.N)), statevec(2, FiniteBasis(2))))
    return full(H), c_op_list, psi*psi', a*a'
end

operator_set (generic function with 1 method)

In [7]:
Base.start(p::SampleProc, cfg::JaynesCummings) = (operator_set(p, cfg))

start (generic function with 37 methods)

In [8]:
Base.length(p::SampleProc, cfg::JaynesCummings) = cfg.N

length (generic function with 50 methods)

In [9]:
function Base.run{Alg}(p::SampleProc{Alg}, cfg::JaynesCummings, s::(QuBase.QuArray, Array, QuBase.QuArray, QuBase.QuArray))
    qprop = QuPropagator(s[1], s[2], s[3], cfg.tlist, Alg())
    for (t, psi) in qprop
        trace(coeffs(psi*(s[4])))
    end
end

run (generic function with 4 methods)

In [10]:
Base.done(p::SampleProc, cfg, s) = nothing

done (generic function with 40 methods)

In [11]:
procs = Proc[ SampleProc{QuODE45}(),
    SampleProc{QuODE78}()]
              #SampleProc{QuODE23s}()]

2-element Array{Proc,1}:
 SampleProc{QuODE45}()
 SampleProc{QuODE78}()

In [12]:
# Setting up various configurations.
cfgs = JaynesCummings[JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(20, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(25, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.:0.1:2*pi),
                      JaynesCummings(20, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.:0.1:2*pi),
                      JaynesCummings(25, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.:0.1:2*pi),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(20, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(25, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(20, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(25, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0)]

12-element Array{JaynesCummings,1}:
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                  
 JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                  
 JaynesCummings(25,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                  
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                 
 JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                 
 JaynesCummings(25,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                 
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                  
 JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.20134228187919462:30.0) 
 Jay

In [13]:
rtable = run(procs, cfgs)

Benchmarking quode45 ...
  quode45 with cfg = JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 1, elapsed = 1.201556765 secs
  quode45 with cfg = JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 1, elapsed = 18.540501571 secs
  quode45 with cfg = JaynesCummings(25,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 1, elapsed = 43.771175868 secs
  quode45 with cfg = JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2): nruns = 1, elapsed = 1.259153078 secs
  quode45 with cfg = JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2): nruns = 1, elapsed = 18.537888301 secs
  quode45 with cfg = JaynesCummings(25,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2): nruns = 1, elapsed = 44.234999907 secs
  

BenchmarkTable [unit = sec]
config                                                                                                                 |  quode45  quode78  
--------------------------------------------------------------------------------------------------------------------------------------------
JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   |   1.2016   1.1642  
JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   |  18.5405  17.4558  
JaynesCummings(25,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   |  43.7712  41.9951  
JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  |   1.2592   1.4124  
JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  |  18.5

In [14]:
show(rtable)

BenchmarkTable [unit = sec]
config                                                                                                                 |  quode45  quode78  
--------------------------------------------------------------------------------------------------------------------------------------------
JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   |   1.2016   1.1642  
JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   |  18.5405  17.4558  
JaynesCummings(25,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   |  43.7712  41.9951  
JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  |   1.2592   1.4124  
JaynesCummings(20,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  |  18.5